#팀 멤버

- 20186030 김재윤
- 20185296 원동연
- 20186889 권용한

#실험 Dataset

Spaceship Titanic

#실험의 목적

현재 데이터 셋에서 불필요한 데이터들도 섞여있고, 여러 데이터가 합쳐져 하나의 데이터로 제공되기도 하며, 유사한 데이터들이 구분되어 제공되기도 합니다.
또한 제공되는 데이터 중 null값으로 제공되는 경우도 존재하게 됩니다.
때문에 더 효과적인 연산 및 예측을 위해선 데이터 전처리 과정이 필수적으로 수반되어야 합니다.
현재 하고자 하는 내용이 연속적인 값을 계산하는 것이 아닌 binary classification을 필요로 하고 있기에, 이전의 Linear regression 보다는 NN, NB, SVM, DT와 같은 방법이 더 효과적으로 동작하게 될 것으로 생각합니다.
각 방법마다 scaling 기법의 중요도도 달라질 수 있을 것이며, 중요도가 높다고 여겨지는 데이터의 종류도 달라질 것으로 생각합니다.
때문에 유의미한 결과를 도출하려면 이러한 점을 고려해 기법마다 효과적인 데이터 관리를 필요로 하며, 이를 실험함으로써 기법마다 중요하다고 판단되는 data에 대한 insight가 증진 될 것으로 생각하며, 각 기법의 동작 메커니즘에 대한 이해도를 높일 수 있다고 생각합니다.


#개선을 위한 목적

성공적인 알고리즘의 수행을 위해선 데이터 전처리 과정이 필수적입니다.
우선 데이터 중에서 현재 NULL값이 존재하기 때문에, 해당 데이터에 대해서 적절한 데이터의 할당을 필요로 합니다.
우선 수치 데이터에 대해선 중간값을, string 데이터에 대해선 최빈값을 우선 적용하는 것이 좋다고 판단했습니다.
복합 데이터로 제공되는 데이터의 경우엔 string split을 통해 데이터를 분리 후 필요한 데이터를 사용했으며,필요에 따라 수치 데이터 연산을 통해 feature을 줄이면서 효과적인 결과를 도출할 필요가 있다고 생각합니다.

해당 과정에 있어서는, 하이퍼 파라미터에 대한 조정 및 어떤 커널을 사용해야 하는 지와 LinearSVC의 경우에 있어서는 어떤 Regularization을 적용할 것인지가 굉장히 중요하다. 따라서 이에 대한 조정이 필수적으로 전제가 된다.

현재 해당 Test의 경우에 있어서는 Missing Value를 임의 값으로 대입을 해놓았기에, 최빈값 혹은 평균값에 대한 적용을 하는 것이 중요하다고 생각합니다.

In [225]:
from google.colab import drive
import os, sys
import pandas as pd
import numpy as np
drive.mount('gdrive', force_remount=True)
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import accuracy_score

Mounted at gdrive


# Data Reading By pandas

train 과 test Data의 경우 따로 csv 파일로 존재.

In [226]:
file_path = '/content/gdrive/MyDrive/Colab Notebooks/titanic/'
df_train = pd.read_csv(file_path + 'train.csv')
df_test = pd.read_csv(file_path + 'test.csv')

df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [227]:
df_train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


#Missing Data Processing.

In [228]:
df_train[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = df_train[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].fillna(0)
df_test[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = df_test[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].fillna(0)

df_train['Age'] =df_train['Age'].fillna(df_train['Age'].median())
df_test['Age'] =df_test['Age'].fillna(df_test['Age'].median())

df_train['VIP'] =df_train['VIP'].fillna(False)
df_test['VIP'] =df_test['VIP'].fillna(False)

df_train['HomePlanet'] =df_train['HomePlanet'].fillna('Mars')
df_test['HomePlanet'] =df_test['HomePlanet'].fillna('Mars')

df_train['Destination']=df_train['Destination'].fillna("PSO J318.5-22")
df_test['Destination']=df_test['Destination'].fillna("PSO J318.5-22")

df_train['CryoSleep'] =df_train['CryoSleep'].fillna(False)
df_test['CryoSleep'] =df_test['CryoSleep'].fillna(False)

df_train['Cabin'] =df_train['Cabin'].fillna('T/0/P')
df_test['Cabin'] =df_test['Cabin'].fillna('T/0/P')

In [229]:
df_train[['Deck','Num','Side']] = df_train.Cabin.str.split('/',expand=True)
df_test[['Deck','Num','Side']] = df_test.Cabin.str.split('/',expand=True)
df_train['PassengerGroup'] = df_train.PassengerId.str.split('_', expand = True)[0]
df_test['PassengerGroup'] = df_train.PassengerId.str.split('_', expand = True)[0]

In [230]:
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,PassengerGroup
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P,0001
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S,0002
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S,0003
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S,0003
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S,0004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,A,98,P,9276
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,G,1499,S,9278
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,G,1500,S,9279
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,E,608,S,9280


#Processing String Label


In [231]:
categorical_cols= ['HomePlanet','CryoSleep','Destination','VIP','Deck','Side','Num', 'PassengerGroup']
for i in categorical_cols:
    print(i)
    le=LabelEncoder()
    arr=np.concatenate((df_train[i], df_test[i])).astype(str)
    le.fit(arr)
    df_train[i]=le.transform(df_train[i].astype(str))
    df_test[i]=le.transform(df_test[i].astype(str))

HomePlanet
CryoSleep
Destination
VIP
Deck
Side
Num
PassengerGroup


# Train Test Split (7 : 3) 

In [232]:
df_train= df_train.drop(['Name','Cabin', 'PassengerId'],axis=1)
df_test= df_test.drop(['Name','Cabin', 'PassengerId'],axis=1)

In [233]:
df_train['Transported']=df_train['Transported'].replace({True:1,False:0})

In [234]:
df_train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,PassengerGroup
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,1,5,0,1,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,0,0,1,2
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,0,0,1,2
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,1,5,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0,0,1872,0,6213
8689,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,0,6,556,1,6214
8690,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,1,6,559,1,6215
8691,1,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0,4,1460,1,6216


#Train Test Split (7 : 3)

In [235]:
X = df_train.drop('Transported', axis = 1)
y = df_train['Transported']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# K-nearest Neighbor test : All Hyper-parameter is Default Value.

In [236]:
knn = KNeighborsClassifier();

knn.fit(X_train, y_train);

y_pred = knn.predict(X_test);

print(accuracy_score(y_test.values, y_pred));

0.7530674846625767


#Gaussian Naive Bayes test : All Hyper-parameter is Default Value.

In [237]:
NB = GaussianNB();

NB.fit(X_train, y_train);

y_pred = NB.predict(X_test);

print(accuracy_score(y_test.values, y_pred));

0.7039877300613497


#Linear Support Vector Classifier test : All Hyper-parameter is Default Value.

In [238]:
linSVC = LinearSVC()

linSVC.fit(X_train, y_train)

y_pred = linSVC.predict(X_test)

print(accuracy_score(y_test.values, y_pred))

0.6376533742331288


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#SVM with Quadratic equation test : All Hyper-parameter is Default Value


In [239]:
svc = SVC()

svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print(accuracy_score(y_test.values, y_pred))

0.776840490797546


#Decision tree test : All Hyper-parameter is Default Value

In [240]:
tree = DecisionTreeClassifier()

tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

print(accuracy_score(y_test.values, y_pred))

0.7361963190184049


#각 4가지의 알고리즘 동작 시 결과

강의에서 배운 4가지 알고리즘에 대한 모듈은 sklearn에 잘 구현되어 있어 공식 문서를 참고해 모든 parameter를 default로 한 모델을 baseline 모델로 실험을 진행했습니다.

각 모델에 대한 결과는 다음과 같습니다.

K = 5인 KNN 모델 : 0.7530

Guassian Naive bayes 모델 : 0.7039

Linear Support vector 모델 : 0.63

Quadratic Support vector 모델 : 0.77

Decision Tree 모델 : 0.7361

각 모델은 모두 일반적으로 0.7 포인트의 성능을 보여주는 것을 확인할 수 있습니다. 

또한 실습 도중 Group Id라는 feature를 넣으면 오히려 성능이 낮아지는 것을 확인할 수 있었는데 이를 통해 feature selection의 중요성과 모델에 따라 feature의 분류 결과가 달라져 feature의 중요도가 다르다는 것을 알 수 있습니다. 따라서 feature 조합X모델X(hyper parameter)에 따라 가장 높은 성능을 보여주는 모델을 추후 생성할 예정입니다.